In [1]:
import keras
keras.__version__

Using TensorFlow backend.


'2.1.0'

In [2]:
max_words = 10000
maxlen = 30
embedding_dims = 100

### 训练一个双向LSTM模型

In [3]:
#【样本，每个样本30单词，每个单词100维度】
from keras import layers
from keras.models import Sequential


# 定义模型
model = Sequential()
model.add(layers.Embedding(max_words, embedding_dims, input_length=maxlen))#构建词嵌入，每个单词
model.add(layers.Bidirectional(layers.LSTM(32)))
#model.add(layers.Flatten())# 3 -> 2
model.add(layers.Dense(32, activation='relu'))#负责单词之间的练习和语义
model.add(layers.Dense(3, activation='softmax'))
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 100)           1000000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 64)                34048     
_________________________________________________________________
dense_1 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 99        
Total params: 1,036,227
Trainable params: 1,036,227
Non-trainable params: 0
_________________________________________________________________


### 训练并评估一个使用dropout正则化的堆叠LSTM模型(/GRU)

In [4]:
#【样本，每个样本30单词，每个单词100维度】
from keras import layers
from keras.models import Sequential


# 定义模型
model_1 = Sequential()
model_1.add(layers.Embedding(max_words, embedding_dims, input_length=maxlen))#构建词嵌入，每个单词
model_1.add(layers.LSTM(32,
                      dropout=0.1,
                      recurrent_dropout=0.5,
                      return_sequences=True,
                     ))
model_1.add(layers.LSTM(64,
                     activation='relu',
                      dropout=0.1,
                      recurrent_dropout=0.5,
                     ))
#model.add(layers.Flatten())# 3 -> 2
model_1.add(layers.Dense(32, activation='relu'))#负责单词之间的练习和语义
model_1.add(layers.Dense(3, activation='softmax'))
model_1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 100)           1000000   
_________________________________________________________________
lstm_2 (LSTM)                (None, 30, 32)            17024     
_________________________________________________________________
lstm_3 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_3 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_4 (Dense)              (None, 3)                 99        
Total params: 1,044,035
Trainable params: 1,044,035
Non-trainable params: 0
_________________________________________________________________


## Attention_CNN_LSTM

In [17]:
from keras.models import Sequential
from keras import layers


filters = 128
kernel_size = 3

right_branch = Sequential()
right_branch.add(layers.Embedding(max_words, embedding_dims, input_length=maxlen))
right_branch.add(layers.Conv1D(filters=filters,
                                    kernel_size=kernel_size,
                                    padding='valid',
                                    activation='relu',
                                     ))
right_branch.add(layers.MaxPooling1D(3))


left_branch = Sequential()
left_branch.add(layers.Embedding(max_words, embedding_dims, input_length=maxlen))
left_branch.add(layers.LSTM(128, return_sequences=True))

merged = layers.Merge([left_branch, right_branch], mode='dot', output_shape=lambda x: x[0])

final_model = Sequential()
final_model.add(merged)
final_model.add(layers.GlobalAveragePooling1D())
final_model.add(layers.Dense(3, activation='softmax'))

final_model.summary()
#right_branch.summary()

#model.add(layers.MaxPooling1D(3))

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_11 (Merge)             (None, 30, 9)             0         
_________________________________________________________________
global_average_pooling1d_2 ( (None, 9)                 0         
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 30        
Total params: 2,155,806
Trainable params: 2,155,806
Non-trainable params: 0
_________________________________________________________________


F:\Anaconda\envs\tensorflow-gpu\lib\site-packages\ipykernel_launcher.py:22: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
